In [23]:
!pip3 install -r requirements.txt
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 432 kB 4.3 MB/s 
     |████████████████████████████████| 13.7 MB 1.6 MB/s 
     |████████████████████████████████| 3.5 MB 35.4 MB/s 
     |████████████████████████████████| 1.2 MB 42.8 MB/s 
     |████████████████████████████████| 622 kB 47.3 MB/s 
     |████████████████████████████████| 351 kB 39.5 MB/s 
     |████████████████████████████████| 89 kB 6.8 MB/s 
     |████████████████████████████████| 6.0 MB 32.0 MB/s 
     |████████████████████████████████| 1.6 MB 23.5 MB/s 
     |████████████████████████████████| 181 kB 53.0 MB/s 
     |████████████████████████████████| 10.1 MB 41.2 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 628 kB 47.1 MB/s 
     |████████████████████████████████| 451 kB 43.1 MB/s 
     |████████████████████████████████| 174 kB 57.9 MB/s 
     |████████████████████████████████| 5.9 MB 27.7 MB/s 
     |████████████████████████████████| 895 kB 52.9 MB/s 
     |████████████

#Datos WhatsApp

In [4]:
#Datos bd
import pandas as pd

df_msg = pd.read_csv('Message.csv')
df_cat = pd.read_csv('Category.csv')
df_user = pd.read_csv('User.csv') 

df_userMes = pd.read_csv('UserMessage.csv') 
df_mesCat = pd.read_csv('MessageCategory.csv') 

In [5]:
print(df_msg.shape,df_cat.shape, df_user.shape,df_userMes.shape,df_mesCat.shape)

(112821, 8) (550, 10) (32805, 2) (680000, 2) (955203, 2)


In [6]:
df_msg= df_msg.dropna(subset=['data_pp'])

In [7]:
final = pd.merge(df_msg, df_mesCat, on='id_message')
final = pd.merge(final, df_cat, on='id_category')

In [8]:
df = final.copy()
df = df.drop(columns=['id_message', 'dataset','timestamp','data','id_forwarded','emojis','id_reply'])

In [9]:
df

,data_pp,id_category,name,target,is_general,is_women,is_union,is_geographic,is_thematic,deleted,hacked
0,persona 3ro edad x provoste diciembre x kast p...,0,Atrévete con kast🇨🇱🗣️,Kast,True,False,False,False,False,False,False
1,mismo cecilia comunismo pobres cubano,0,Atrévete con kast🇨🇱🗣️,Kast,True,False,False,False,False,False,False
2,siiiiiiiiiiiiiiii,0,Atrévete con kast🇨🇱🗣️,Kast,True,False,False,False,False,False,False
3,parisi,0,Atrévete con kast🇨🇱🗣️,Kast,True,False,False,False,False,False,False
4,boric,0,Atrévete con kast🇨🇱🗣️,Kast,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
92169,insitu apoderado titular general jefe local ap...,65,Bulnes por Boric,Boric,False,False,False,True,False,False,False
92170,apoderado general,65,Bulnes por Boric,Boric,False,False,False,True,False,False,False
92171,mesa santa cruz,65,Bulnes por Boric,Boric,False,False,False,True,False,False,False
92172,pal pal,65,Bulnes por Boric,Boric,False,False,False,True,False,False,False


In [10]:
df = df.drop_duplicates(subset=['data_pp'])

#Working with a dictionary dataset

In [11]:
import re
def remove(list):
    pattern = '[0-9]'
    list = [re.sub(pattern, '', i) for i in list]
    return list

In [12]:
def has_numbers(inputString):
  return any(char.isdigit() for char in inputString)

In [13]:
from unicodedata import normalize

def remove_tilde(list):
  list = [normalize('NFC',re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize( "NFD", i), 0, re.I)) for i in list]
  return list

In [14]:
def minuscula(list):
  list = [i.lower() for i in list]
  return list

In [15]:
import zipfile
zipf = zipfile.ZipFile('dict_rae_txt.zip')
zipf.extractall()

In [16]:
diccionario_sp=[]; d_sp=[]

In [17]:
import os
for i in [name for name in os.listdir('dics') if os.path.isfile(os.path.join('dics', name))]:
  with open('dics/'+i, 'r', encoding="utf8") as reader:
    for line in reader.readlines():
      diccionario_sp.append(line.rstrip())


In [18]:
diccionario_sp = remove(diccionario_sp)
diccionario_sp = remove_tilde(diccionario_sp)
diccionario_sp =list(filter(lambda a: a.find('-') < 0, diccionario_sp))
diccionario_sp = minuscula(diccionario_sp)

In [ ]:
for j in [row for row in diccionario_sp if len(row.split(",")) > 1]:
    a,b=j.split(',')
    b = b.replace(" ", "")
    d_sp.append(a)
    d_sp.append(a[:len(a)-len(b)]+b)
    
    

[d_sp.append(row) for row in diccionario_sp if len(row.split(",")) <= 1]

In [20]:
d_sp = pd.DataFrame(d_sp, columns=['palabras'])
d_sp = d_sp.drop_duplicates()
d_sp.shape

(105052, 1)

In [21]:
d_sp

,palabras
0,wagneriano
1,wagneriana
2,washingtoniano
3,washingtoniana
4,weimares
...,...
109528,xinca
109529,xiote
109530,xochil
109531,xocoatole


#Frequent words WSP

In [24]:
from autocorrect import Speller
spell = Speller('es')

from tqdm import tqdm

def corrector_py(list):
  list = [spell(i) for i in tqdm(list)]
  return list

dictionary for this language not found, downloading...
__________________________________________________
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
done!


corrected_with_speller is the dataset after using corrector_py

In [50]:
saved = pd.read_csv('corrected_with_speller.csv',index_col=['Unnamed: 0'])
saved=saved.rename(columns={"0": "chat"})
saved

,chat
0,persona 3ro edad x provost diciembre x kast pa...
1,mismo cecilia comunismo pobres cubano
2,siiiiiiiiiiiiiiii
3,parisi
4,boris
...,...
406335,compita pte rojo
406336,connie
406337,min tiempo
406338,insurgencia anti comunista


In [27]:
import gensim
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(saved['0'].tolist())
freq = np.ravel(X.sum(axis=0))

In [30]:
import operator

vocab = [v[0] for v in sorted(vectorizer.vocabulary_.items(), key=operator.itemgetter(1))]
fdist = dict(zip(vocab, freq))

In [31]:
from pprint import pprint
sorted_d = dict( sorted(fdist.items(), key=operator.itemgetter(1),reverse=True))

print('Orden descendiente : ',sorted_d)
print('Cantidad de palabras del corpus:', len(sorted_d))

Orden descendiente :  {'00': 61698, 'boris': 31939, 'type': 25937, 'home': 23839, 'adr': 23767, 'kast': 23765, 'gente': 18365, 'grupo': 15739, 'dia': 15132, 'douglasmodder': 14880, 'voto': 14644, 'hola': 13295, 'mesa': 13223, 'apoderado': 13137, 'persona': 12828, 'puerta': 11856, 'chile': 11494, 'campaña': 11422, 'whatsapp': 10388, 'work': 9140, 'bf': 8912, 'net': 8893, 'bd': 8884, '67': 8855, 'buen': 8523, 'favor': 7669, 'candidato': 7613, 'presidente': 7168, 'hora': 7156, 'plaza': 7044, 'programa': 6897, 'actividad': 6860, 'solo': 6827, 'comando': 6805, 'idea': 6708, '01': 6643, 'ef': 6643, '4f': 6639, 'tema': 6555, 'bandera': 6539, 'cosa': 6469, 'partido': 6367, 'pais': 6321, 'mejor': 6272, 'gracias': 6232, 'gabriel': 6096, 'domingo': 5987, 'importante': 5801, 'jajaja': 5677, 'vez': 5589, 'bueno': 5315, 'reunion': 5077, 'vuelta': 5040, 'año': 4969, 'red': 4961, 'nuevo': 4821, 'trabajo': 4782, 'social': 4775, 'video': 4761, 'spammer': 4757, 'derecha': 4712, 'primero': 4712, 'parisi':

# Chilean dataset

In [32]:
dataset_ch = [(x,sorted_d.get(x)) for x in list(sorted_d.keys()) if x not in d_sp.palabras.tolist()]

In [36]:
dataset_ch

[('00', 61698),
 ('boris', 31939),
 ('type', 25937),
 ('home', 23839),
 ('adr', 23767),
 ('kast', 23765),
 ('douglasmodder', 14880),
 ('whatsapp', 10388),
 ('work', 9140),
 ('bf', 8912),
 ('net', 8893),
 ('bd', 8884),
 ('67', 8855),
 ('01', 6643),
 ('ef', 6643),
 ('4f', 6639),
 ('gracias', 6232),
 ('gabriel', 6096),
 ('jajaja', 5677),
 ('spammer', 4757),
 ('parisi', 4679),
 ('53', 4437),
 ('61', 4432),
 ('6f', 4428),
 ('pdg', 4287),
 ('douglaskjkjkjk', 4125),
 ('link', 4041),
 ('55988220785', 3237),
 ('vcard', 3228),
 ('servel', 2950),
 ('twitter', 2720),
 ('20', 2556),
 ('54', 2290),
 ('13', 2271),
 ('2a', 2252),
 ('d9', 2235),
 ('d3', 2233),
 ('d5', 2233),
 ('64', 2227),
 ('d4', 2224),
 ('02', 2223),
 ('48', 2222),
 ('75', 2221),
 ('06', 2220),
 ('68', 2219),
 ('70', 2219),
 ('72', 2219),
 ('73', 2218),
 ('79', 2218),
 ('65', 2216),
 ('c1', 2216),
 ('6e', 2215),
 ('c0', 2214),
 ('c6', 2214),
 ('3e', 2211),
 ('967', 2057),
 ('968', 2053),
 ('hrs', 1984),
 ('info', 1912),
 ('proximo', 

In [37]:
df_dataset_ch = pd.DataFrame(dataset_ch, columns=['palabras','freq'])
print('Palabras chilenas que no aparecen en el diccionario: ',df_dataset_ch.shape[0])

Palabras chilenas que no aparecen en el diccionario:  56812


In [ ]:
len(df_dataset_ch['palabras'].tolist())

56812

df_dataset_ch contiene palabras que no aparecen en el diccionario 

In [38]:
df_dataset_ch

,palabras,freq
0,00,61698
1,boris,31939
2,type,25937
3,home,23839
4,adr,23767
...,...,...
56807,𝚜𝚘𝚢,1
56808,𝚝𝚘𝚍𝚘,1
56809,𝚝𝚘𝚝𝚊𝚕,1
56810,𝚞𝚗,1


df_dataset_ch contiene palabras que no aparecen en el diccionario y no contienen números

In [40]:
dt_ch = [ (df_dataset_ch['palabras'].tolist()[i],df_dataset_ch['freq'].tolist()[i]) for i in range(len(df_dataset_ch['palabras'].tolist())) if has_numbers(df_dataset_ch['palabras'].tolist()[i]) is False]
df_dataset_ch = pd.DataFrame(dt_ch, columns=['palabras','freq'])
print('Palabras chilenas que no aparecen en el diccionario y no contienen números: ',df_dataset_ch.shape[0])

Palabras chilenas que no aparecen en el diccionario y no contienen números:  42773


In [41]:
df_dataset_ch

,palabras,freq
0,boris,31939
1,type,25937
2,home,23839
3,adr,23767
4,kast,23765
...,...,...
42768,𝚜𝚘𝚢,1
42769,𝚝𝚘𝚍𝚘,1
42770,𝚝𝚘𝚝𝚊𝚕,1
42771,𝚞𝚗,1


In [55]:
df_dataset_ch.to_csv('palabras_chilenas.csv')

# FastText

Este word embedding sirve para corregir palabras del dataset, sin embargo esta tarea es costosa y no puede ser llevada a cabo totalmente por un modelo de Machine Learning. Por lo que se recomienda escoger las palabras más frequentes del dataset que no tiene palabras identificables en el diccionario (df_dataset_ch) y corregirlas con la función corregir. Esta función aplica los cambios a todo el dataset y re entrena el embedding.

In [43]:
from gensim.models.phrases import Phrases, Phraser
data = df.data_pp.tolist()
sent = [row.split() for row in data]
phrases = Phrases(sent, min_count=10, progress_per=10000)
bigram = Phraser(phrases)
sentences = bigram[sent]

In [44]:
from gensim.models import FastText
model_ft = FastText(size=300, window=3, min_count=2)  
model_ft.build_vocab(sentences=sentences)
model_ft.train(sentences=sentences, total_examples=model_ft.corpus_count, epochs=10) 

Variaciones de kast que queremos cambiar

In [51]:
model_ft.wv.most_similar(positive=["kast"])

[('kastt', 0.9861874580383301),
 ('kkast', 0.9703152179718018),
 ('kash', 0.9680866003036499),
 ('kakast', 0.9678688049316406),
 ('\U0001f7e0kast', 0.9673939943313599),
 ('jkast', 0.9650557041168213),
 ('kkkast', 0.9625639319419861),
 ('jakast', 0.9545525312423706),
 ('j.a.kast', 0.951275110244751),
 ('kass', 0.950474739074707)]

In [52]:
def corregir(palabra, corpus, threshold= 0.88, train= False, model=model_ft):
  v = [i[0] for i in model.wv.most_similar(positive=[palabra],topn=100) if i[1]> threshold]

  for i in v:
    corpus['chat'] = corpus['chat'].replace({i: palabra}, regex=True)
  
  if train is True:
    sent = [row.split() for row in corpus.chat.tolist()]
    phrases = Phrases(sent, min_count=10, progress_per=10000)
    bigram = Phraser(phrases)
    sentences = bigram[sent]
    model = FastText(size=300, window=3, min_count=2)  
    model.build_vocab(sentences=sentences)
    model.train(sentences=sentences, total_examples=model.corpus_count, epochs=10) 
    
  return corpus,model

Ejemplo: corregir las variaciones de "kast"

In [53]:
corpus,modelo = corregir(palabra='kast', corpus=saved, train=True, model=model_ft)

In [54]:
modelo.wv.most_similar(positive=["kast"]) #se puede seguir corrigiendo o se puede escoger un threshold más bajo. (las palabras anteriores no aparecen)

[('kauskast', 0.9559367895126343),
 ('kaste', 0.873738169670105),
 ('kastakak', 0.8719120621681213),
 ('kastsjs', 0.8593675494194031),
 ('kasto', 0.8455103635787964),
 ('kakastt', 0.8398998975753784),
 ('akast', 0.8339234590530396),
 ('kastakaka', 0.821984589099884),
 ('kakakastt', 0.8088088035583496),
 ('todoskakastt', 0.8064252138137817)]